# LabelSmoothing v2

In [ ]:
#hide
!fusermount -u drive
!google-drive-ocamlfuse drive
from google.colab import drive
drive.mount('/content/drive')

fusermount: failed to unmount /content/drive: No such file or directory
/bin/bash: google-drive-ocamlfuse: command not found
Mounted at /content/drive


In [ ]:
#!pip install torch
#!pip install pytorch
#!pip install torchvision
#!pip install fastai
!pip3 install git+https://github.com/fastai/fastai.git 
!pip install -Uqq fastbook

  Cloning https://github.com/fastai/fastai.git to /tmp/pip-req-build-u03iop5g
  Running command git clone -q https://github.com/fastai/fastai.git /tmp/pip-req-build-u03iop5g
     |████████████████████████████████| 61kB 5.5MB/s 
  Created wheel for fastai: filename=fastai-2.3.1-cp37-none-any.whl size=193488 sha256=55ea25b7274ed51c772089493674963b6baa3f17da2b63bc81944c15630d8224
  Stored in directory: /tmp/pip-ephem-wheel-cache-apdmjdk1/wheels/cf/46/39/b2d08762125ed2376861976ab2c4ac30c029b86e375735d9b8
Successfully built fastai
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
     |████████████████████████████████| 727kB 9.1MB/s 
     |████████████████████████████████| 1.2MB 48.6MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 


In [ ]:
#hide
import PIL
from PIL import Image 
import os
import requests
import torch
import pandas as pd
import fastbook
from fastai.vision.widgets import *
fastbook.setup_book()

Mounted at /content/gdrive


In [ ]:
#hide
import glob
import requests
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
cwd = os.getcwd()
os.listdir(os.path.join(cwd, ".."))

['tmp',
 'sbin',
 'var',
 'media',
 'root',
 'run',
 'srv',
 'boot',
 'home',
 'mnt',
 'proc',
 'etc',
 'bin',
 'lib64',
 'opt',
 'sys',
 'dev',
 'usr',
 'lib',
 'content',
 '.dockerenv',
 'tools',
 'datalab',
 'tensorflow-1.15.2',
 'lib32']

In [ ]:
#path = Path('gdrive/MyDrive/PBA Softwareudvikling/AI Machinelearning/AI_reCaptcha v2/recaptcha-dataset-main/Large/')
rootPath = glob.glob(os.path.join(os.getcwd(), "gdrive/MyDrive/**/AI_reCaptcha v2/"), recursive=true)[0]
path = os.path.join(rootPath, "recaptcha-dataset-main/Large/")
path

'/content/gdrive/MyDrive/AI_reCaptcha v2/recaptcha-dataset-main/Large/'

In [ ]:
fns = get_image_files(path)
fns

(#0) []

In [ ]:
def get_y(r): return L(parent_label(r))
#Vi prøvede først at lave vores egen måde at loade filer ind for at begrænse datasettet
# def get_image_files_by_size(path, sample_size = 200): 
#   return list(get_image_files(path))[:sample_size]

In [ ]:
dblock = DataBlock(
    blocks = (ImageBlock, MultiCategoryBlock), 
    get_items = get_image_files, 
    get_y = get_y, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms=RandomResizedCrop(128, min_scale=0.35))
dls = dblock.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=200, nrows=10)

In [ ]:
learn = cnn_learner(dls, 
                    resnet50, 
                    metrics=accuracy_multi, 
                    cbs=MixUp)

In [ ]:
lr_min,lr_steep = learn.lr_find()
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
#learn.fine_tune(4, base_lr=2.51e-02)
learn.fine_tune(200, base_lr=lr_min)

In [ ]:
#learn.fit_one_cycle(8.32e-03)

In [ ]:
learnerFile = path = os.path.join(rootPath, "Learner_23-05.pkl")

'/content/gdrive/MyDrive/AI_reCaptcha v2/Learner_04-05.pkl'

In [ ]:
learner = learn.export(fname=learnerFile)

In [ ]:
#learner = load_learner(fname=learnerFile)

In [ ]:
#hide_output
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
img = PILImage.create(btn_upload.data[-1])
#hide_output
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn.predict(img )

#hide_output
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]}'
lbl_pred

In [ ]:
learn.dls.vocab

In [ ]:
numbers = [ f"{x:.4}" for x in probs ]
numbers

In [ ]:
pred_idx